In [1]:
import zipfile
import json
import pandas
# from pathlib import Path
# import threading

In [2]:
ZIP_ARCHIVE = 'egrul.json.zip'
DICT_KEY_SEARCH = 'СвОКВЭД'
OKVED_SEARCH = 'СвОКВЭДОсн'
SV_OKVED_KEY_SEARCH = 'КодОКВЭД'
SV_OKVED_VALUE_SEARCH = '61'
COUNT_OF_FLOW = 1

list_of_company = []

In [3]:
def filter_company_by_okved(source_data):
    return [d for d in source_data if d.get('data',{}).get(DICT_KEY_SEARCH,{}).get(OKVED_SEARCH,{}).get(SV_OKVED_KEY_SEARCH,'')[:2] == SV_OKVED_VALUE_SEARCH]

In [4]:
count = 0;
with zipfile.ZipFile(ZIP_ARCHIVE, 'r') as zip_files:
    for file_name in zip_files.namelist():
        if file_name.endswith('.json'):
            count += 1
        # if file_name == '00001.json':
        # if count < 1000:
            print(f'Обработка файла №{count} имя файла {file_name}')
            with zip_files.open(file_name) as file:
                data = json.load(file)
                data = filter_company_by_okved(data)
                if len (data) > 0:
                    for line in data:
                        list_of_company.append(line)
                    # list_of_company +=filter_company_by_okved(data)

Обработка файла №1 имя файла 06439.json
Обработка файла №2 имя файла 04669.json
Обработка файла №3 имя файла 03534.json
Обработка файла №4 имя файла 00393.json
Обработка файла №5 имя файла 07388.json
Обработка файла №6 имя файла 10339.json
Обработка файла №7 имя файла 09162.json
Обработка файла №8 имя файла 05575.json
Обработка файла №9 имя файла 06849.json
Обработка файла №10 имя файла 08312.json
Обработка файла №11 имя файла 05296.json
Обработка файла №12 имя файла 08667.json
Обработка файла №13 имя файла 10549.json
Обработка файла №14 имя файла 10851.json
Обработка файла №15 имя файла 03216.json
Обработка файла №16 имя файла 07553.json
Обработка файла №17 имя файла 10245.json
Обработка файла №18 имя файла 08935.json
Обработка файла №19 имя файла 00981.json
Обработка файла №20 имя файла 07602.json
Обработка файла №21 имя файла 07293.json
Обработка файла №22 имя файла 08377.json
Обработка файла №23 имя файла 10193.json
Обработка файла №24 имя файла 08450.json
Обработка файла №25 имя ф

In [5]:
df = pandas.DataFrame(list_of_company)
df['okved'] = df['data'].apply(lambda x: x[DICT_KEY_SEARCH])\
    .apply(lambda y: y[OKVED_SEARCH])\
    .apply(lambda z: z[SV_OKVED_KEY_SEARCH])
df = df[['ogrn','inn','kpp','name','full_name','okved']]
    

In [6]:
df

,ogrn,inn,kpp,name,full_name,okved
0,1114910000480,4909109443,490901001,"ООО ""ИТЛАЙН""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИТЛАЙН""",61.10
1,1127847488053,7811531478,781101001,"ООО ""СМО СИСТЕМА""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СМО ...",61.30
2,1197536005787,7536178239,753601001,"ООО ""ИНТЕР КОД""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ИНТЕ...",61.90
3,1197536005919,7513007283,751301001,"ООО ""ВОНГОЛА""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОНГ...",61.10
4,1197627000691,7604352760,760401001,"ООО ""РЭС""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РИНГ...",61.10
...,...,...,...,...,...,...
20912,1111902001694,1902024322,190201001,"ООО ""САЯНСВЯЗЬ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""САЯН...",61.10.1
20913,1112031001422,2016001613,201601001,"ООО ""АВТО-НАВИГАТОР""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АВТО...",61.1
20914,1112031002038,2014005080,201401001,"ООО ""ГАРАНТТОРГ""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ГАРА...",61.10
20915,1112031003831,2014005877,201401001,"ООО ""РЕАС""","ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""РЕАС""",61.10


Многопоточность

In [ ]:
def parse_file_in_flow(number):
    # count = 0;
    # print(f'Работа потока {number}')
    with zipfile.ZipFile(ZIP_ARCHIVE, 'r') as zip_files:
        for file_name in zip_files.namelist():
            if file_name.endswith('.json'):
                path = Path(file_name)
                if (int(path.stem)%COUNT_OF_FLOW == number):
                    # count += 1
                    # if file_name == '00001.json':
                    # print(f'Обработка файла №{count} имя файла {file_name}')
                    with zip_files.open(file_name) as file:
                        data = json.load(file)
                        data = filter_company_by_okved(data)
                        if len (data) > 0:
                            list_of_company.append(filter_company_by_okved(data))

In [ ]:

def run_threads(count_of_tread):
    threads = [
        threading.Thread(target=parse_file_in_flow, args=(i,))
        for i in range(1, count_of_tread)
    ]
    for thread in threads:
        thread.start()  # каждый поток должен быть запущен
    for thread in threads:
        thread.join()  # дожидаемся исполнения всех потоков

In [ ]:
run_threads(COUNT_OF_FLOW)